#Utilities

In [1]:
import contextlib
import copy
import inspect
from collections import OrderedDict
import torch
import requests

In [2]:
def get_module(model, name):
    """
    Finds the named module within the given model.
    """
    for n, m in model.named_modules():
        if n == name:
            return m
    raise LookupError(name)

def replace_module(model, name, new_module):
    """
    Replaces the named module within the given model.
    """
    if "." in name:
        parent_name, attr_name = name.rsplit(".", 1)
        model = get_module(model, parent_name)
    # original_module = getattr(model, attr_name)
    setattr(model, attr_name, new_module)

class StopForward(Exception):
    """
    If the only output needed from running a network is the retained
    submodule then Trace(submodule, stop=True) will stop execution
    immediately after the retained submodule by raising the StopForward()
    exception.  When Trace is used as context manager, it catches that
    exception and can be used as follows:

    with Trace(net, layername, stop=True) as tr:
        net(inp) # Only runs the network up to layername
    print(tr.output)
    """

    pass

def recursive_copy(x, clone=None, detach=None, retain_grad=None):
    """
    Copies a reference to a tensor, or an object that contains tensors,
    optionally detaching and cloning the tensor(s).  If retain_grad is
    true, the original tensors are marked to have grads retained.
    """
    if not clone and not detach and not retain_grad:
        return x
    if isinstance(x, torch.Tensor):
        if retain_grad:
            if not x.requires_grad:
                x.requires_grad = True
            x.retain_grad()
        elif detach:
            x = x.detach()
        if clone:
            x = x.clone()
        return x
    # Only dicts, lists, and tuples (and subclasses) can be copied.
    if isinstance(x, dict):
        return type(x)({k: recursive_copy(v) for k, v in x.items()})
    elif isinstance(x, (list, tuple)):
        return type(x)([recursive_copy(v) for v in x])
    else:
        assert False, f"Unknown type {type(x)} cannot be broken into tensors."

def invoke_with_optional_args(fn, *args, **kwargs):
    """
    Invokes a function with only the arguments that it
    is written to accept, giving priority to arguments
    that match by-name, using the following rules.
    (1) arguments with matching names are passed by name.
    (2) remaining non-name-matched args are passed by order.
    (3) extra caller arguments that the function cannot
        accept are not passed.
    (4) extra required function arguments that the caller
        cannot provide cause a TypeError to be raised.
    Ordinary python calling conventions are helpful for
    supporting a function that might be revised to accept
    extra arguments in a newer version, without requiring the
    caller to pass those new arguments.  This function helps
    support function callers that might be revised to supply
    extra arguments, without requiring the callee to accept
    those new arguments.
    """
    argspec = inspect.getfullargspec(fn)
    pass_args = []
    used_kw = set()
    unmatched_pos = []
    used_pos = 0
    defaulted_pos = len(argspec.args) - (
        0 if not argspec.defaults else len(argspec.defaults)
    )
    # Pass positional args that match name first, then by position.
    for i, n in enumerate(argspec.args):
        if n in kwargs:
            pass_args.append(kwargs[n])
            used_kw.add(n)
        elif used_pos < len(args):
            pass_args.append(args[used_pos])
            used_pos += 1
        else:
            unmatched_pos.append(len(pass_args))
            pass_args.append(
                None if i < defaulted_pos else argspec.defaults[i - defaulted_pos]
            )
    # Fill unmatched positional args with unmatched keyword args in order.
    if len(unmatched_pos):
        for k, v in kwargs.items():
            if k in used_kw or k in argspec.kwonlyargs:
                continue
            pass_args[unmatched_pos[0]] = v
            used_kw.add(k)
            unmatched_pos = unmatched_pos[1:]
            if len(unmatched_pos) == 0:
                break
        else:
            if unmatched_pos[0] < defaulted_pos:
                unpassed = ", ".join(
                    argspec.args[u] for u in unmatched_pos if u < defaulted_pos
                )
                raise TypeError(f"{fn.__name__}() cannot be passed {unpassed}.")
    # Pass remaining kw args if they can be accepted.
    pass_kw = {
        k: v
        for k, v in kwargs.items()
        if k not in used_kw and (k in argspec.kwonlyargs or argspec.varargs is not None)
    }
    # Pass remaining positional args if they can be accepted.
    if argspec.varargs is not None:
        pass_args += list(args[used_pos:])
    return fn(*pass_args, **pass_kw)



def set_requires_grad(requires_grad, *models):
    """
    Sets requires_grad true or false for all parameters within the
    models passed.
    """
    for model in models:
        if isinstance(model, torch.nn.Module):
            for param in model.parameters():
                param.requires_grad = requires_grad
        elif isinstance(model, (torch.nn.Parameter, torch.Tensor)):
            model.requires_grad = requires_grad
        else:
            assert False, "unknown type %r" % type(model)


In [3]:
class Trace(contextlib.AbstractContextManager):
    """
    To retain the output of the named layer during the computation of
    the given network:

        with Trace(net, 'layer.name') as ret:
            _ = net(inp)
            representation = ret.output

    A layer module can be passed directly without a layer name, and
    its output will be retained.  By default, a direct reference to
    the output object is returned, but options can control this:

        clone=True  - retains a copy of the output, which can be
            useful if you want to see the output before it might
            be modified by the network in-place later.
        detach=True - retains a detached reference or copy.  (By
            default the value would be left attached to the graph.)
        retain_grad=True - request gradient to be retained on the
            output.  After backward(), ret.output.grad is populated.

        retain_input=True - also retains the input.
        retain_output=False - can disable retaining the output.
        edit_output=fn - calls the function to modify the output
            of the layer before passing it the rest of the model.
            fn can optionally accept (output, layer) arguments
            for the original output and the layer name.
        stop=True - throws a StopForward exception after the layer
            is run, which allows running just a portion of a model.
    """

    def __init__(
        self,
        module,
        layer=None,
        retain_output=True,
        retain_input=False,
        clone=False,
        detach=False,
        retain_grad=False,
        edit_output=None,
        stop=False,
    ):
        """
        Method to replace a forward method with a closure that
        intercepts the call, and tracks the hook so that it can be reverted.
        """
        retainer = self
        self.layer = layer
        if layer is not None:
            module = get_module(module, layer)

        def retain_hook(m, inputs, output):
            if retain_input:
                retainer.input = recursive_copy(
                    inputs[0] if len(inputs) == 1 else inputs,
                    clone=clone,
                    detach=detach,
                    retain_grad=False,
                )  # retain_grad applies to output only.
            if edit_output:
                output = invoke_with_optional_args(
                    edit_output, output=output, layer=self.layer
                )
            if retain_output:
                retainer.output = recursive_copy(
                    output, clone=clone, detach=detach, retain_grad=retain_grad
                )
                # When retain_grad is set, also insert a trivial
                # copy operation.  That allows in-place operations
                # to follow without error.
                if retain_grad:
                    output = recursive_copy(retainer.output, clone=True, detach=False)
            if stop:
                raise StopForward()
            return output

        self.registered_hook = module.register_forward_hook(retain_hook)
        self.stop = stop

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.close()
        if self.stop and issubclass(type, StopForward):
            return True

    def close(self):
        self.registered_hook.remove()



In [4]:
class TraceDict(OrderedDict, contextlib.AbstractContextManager):
    """
    To retain the output of multiple named layers during the computation
    of the given network:

        with TraceDict(net, ['layer1.name1', 'layer2.name2']) as ret:
            _ = net(inp)
            representation = ret['layer1.name1'].output

    If edit_output is provided, it should be a function that takes
    two arguments: output, and the layer name; and then it returns the
    modified output.

    Other arguments are the same as Trace.  If stop is True, then the
    execution of the network will be stopped after the last layer
    listed (even if it would not have been the last to be executed).
    """

    def __init__(
        self,
        model,
        layers=None,
        retain_output=True,
        retain_input=False,
        clone=False,
        detach=False,
        retain_grad=False,
        edit_output=None,
        stop=False,
    ):
        self.stop = stop

        def flag_last_unseen(it):
            try:
                it = iter(it)
                prev = next(it)
                seen = set([prev])
            except StopIteration:
                return
            for item in it:
                if item not in seen:
                    yield False, prev
                    seen.add(item)
                    prev = item
            yield True, prev

        for is_last, layer in flag_last_unseen(layers):
            self[layer] = Trace(
                module=model,
                layer=layer,
                retain_output=retain_output,
                retain_input=retain_input,
                clone=clone,
                detach=detach,
                retain_grad=retain_grad,
                edit_output=edit_output,
                stop=stop and is_last,
            )

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.close()
        if self.stop and issubclass(type, StopForward):
            return True

    def close(self):
        for layer, trace in reversed(self.items()):
            trace.close()

#Main

In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00


In [6]:
!pip install stanza
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=c7ed2c912bfa937c12e31e3f6be7eaed5cc8d2c614c6255c256b93f7c0132d10
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.7 MB/s eta 0:00:00


In [7]:
import os, re, json
import torch, numpy
from collections import defaultdict
import numpy as np
import torch
from sklearn.decomposition import PCA
import matplotlib
from matplotlib import pyplot as plt
import itertools
import nltk
import random
# from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import stanza
import datasets
import scipy
import csv
import pandas as pd
from PIL import Image
from typing import Any, Optional, Tuple, Union,OrderedDict

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import BlipForQuestionAnswering,BlipProcessor
model= BlipForQuestionAnswering.from_pretrained('Salesforce/blip-vqa-base')
processor=BlipProcessor.from_pretrained('Salesforce/blip-vqa-base')

model.to(device)

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [10]:
model_name='Salesforce/blip-vqa-base'

In [11]:
class ModelandProcessor:
  def __init__(
        self,
        model_name=None,
        model=None,
        processor=None,
        low_cpu_mem_usage=False,
        torch_dtype=None,
    ):
        if  processor is None:
            assert model_name is not None
            processor = BlipProcessor.from_pretrained(model_name)
        if model is None:
            assert model_name is not None
            model = BlipForQuestionAnswering.from_pretrained(
                model_name, low_cpu_mem_usage=low_cpu_mem_usage, torch_dtype=torch_dtype
            )
            set_requires_grad(False, model)
            model.eval().cuda()
        self.processor = processor
        self.model = model
        self.layer_names = [
            n
            for n, m in model.named_modules()
            if (re.match(r"^(vision_model|text_encoder|text_decoder.bert|text_decoder.cls)\.(embeddings|encoder.layers|encoder.layer|predictions)\.(\d+$)", n))
        ]
        self.num_layers = (len(self.layer_names)//3)

  def __repr__(self):
        return (
            f"ModelAndTokenizer(model: {type(self.model).__name__} "
            f"[{self.num_layers} layers], "
            f"tokenizer: {type(self.tokenizer).__name__})"
        )


In [12]:
mt=ModelandProcessor(model_name,model,processor)

In [ ]:
(mt.num_layers)

12

In [13]:
def layername(model, num,block_name, kind=None):
  if block_name=="text_encoder":
    if hasattr(model,"model_text_enc"):
        if kind == "embed":
            return "model_text_enc.embeddings"
        return f'model_text_enc.encoder.layer.{num}{"" if kind is None else "." + kind}'
  elif block_name=="text_decoder":
    if hasattr(model,"model_text_dec"):
        if kind == "embed":
            return f'model_text_dec.bert.embeddings'
        elif kind=="cls.decoder":
            return f'text_decoder.cls.predictions.decoder'
        return f'model_text_dec.bert.encoder.layer.{num}{"" if kind is None else "." + kind}'
    assert False, "unknown transformer structure"

In [ ]:
_from transformers import BlipConfig

In [14]:
class ModelOutput(OrderedDict):
    """
    Base class for all model outputs as dataclass. Has a `__getitem__` that allows indexing by integer or slice (like a
    tuple) or strings (like a dictionary) that will ignore the `None` attributes. Otherwise behaves like a regular
    python dictionary.

    <Tip warning={true}>

    You can't unpack a `ModelOutput` directly. Use the [`~utils.ModelOutput.to_tuple`] method to convert it to a tuple
    before.

    </Tip>
    """

    def __post_init__(self):
        class_fields = fields(self)

        # Safety and consistency checks
        if not len(class_fields):
            raise ValueError(f"{self.__class__.__name__} has no fields.")
        if not all(field.default is None for field in class_fields[1:]):
            raise ValueError(f"{self.__class__.__name__} should not have more than one required field.")

        first_field = getattr(self, class_fields[0].name)
        other_fields_are_none = all(getattr(self, field.name) is None for field in class_fields[1:])

        if other_fields_are_none and not is_tensor(first_field):
            if isinstance(first_field, dict):
                iterator = first_field.items()
                first_field_iterator = True
            else:
                try:
                    iterator = iter(first_field)
                    first_field_iterator = True
                except TypeError:
                    first_field_iterator = False

            # if we provided an iterator as first field and the iterator is a (key, value) iterator
            # set the associated fields
            if first_field_iterator:
                for idx, element in enumerate(iterator):
                    if (
                        not isinstance(element, (list, tuple))
                        or not len(element) == 2
                        or not isinstance(element[0], str)
                    ):
                        if idx == 0:
                            # If we do not have an iterator of key/values, set it as attribute
                            self[class_fields[0].name] = first_field
                        else:
                            # If we have a mixed iterator, raise an error
                            raise ValueError(
                                f"Cannot set key/value for {element}. It needs to be a tuple (key, value)."
                            )
                        break
                    setattr(self, element[0], element[1])
                    if element[1] is not None:
                        self[element[0]] = element[1]
            elif first_field is not None:
                self[class_fields[0].name] = first_field
        else:
            for field in class_fields:
                v = getattr(self, field.name)
                if v is not None:
                    self[field.name] = v

    def __delitem__(self, *args, **kwargs):
        raise Exception(f"You cannot use ``__delitem__`` on a {self.__class__.__name__} instance.")

    def setdefault(self, *args, **kwargs):
        raise Exception(f"You cannot use ``setdefault`` on a {self.__class__.__name__} instance.")

    def pop(self, *args, **kwargs):
        raise Exception(f"You cannot use ``pop`` on a {self.__class__.__name__} instance.")

    def update(self, *args, **kwargs):
        raise Exception(f"You cannot use ``update`` on a {self.__class__.__name__} instance.")

    def __getitem__(self, k):
        if isinstance(k, str):
            inner_dict = dict(self.items())
            return inner_dict[k]
        else:
            return self.to_tuple()[k]

    def __setattr__(self, name, value):
        if name in self.keys() and value is not None:
            # Don't call self.__setitem__ to avoid recursion errors
            super().__setitem__(name, value)
        super().__setattr__(name, value)

    def __setitem__(self, key, value):
        # Will raise a KeyException if needed
        super().__setitem__(key, value)
        # Don't call self.__setattr__ to avoid recursion errors
        super().__setattr__(key, value)

    def to_tuple(self) -> Tuple[Any]:
        """
        Convert self to a tuple containing all the attributes/keys that are not `None`.
        """
        return tuple(self[k] for k in self.keys())

In [15]:
class BlipVQAOutput(ModelOutput):
    """
    Adapted from the base class for vision model's outputs that also contains image embeddings of the pooling of the
    last hidden states. This class also adds the loss term from the text decoder.

    Args:
        loss (`torch.FloatTensor` of shape `(1,)`, *optional*, returned when `labels` is provided):
            Languge modeling loss from the text decoder.
        image_embeds (`torch.FloatTensor` of shape `(batch_size, output_dim)` *optional* returned when model is initialized with `with_projection=True`):
            The image embeddings obtained by applying the projection layer to the pooler_output.
        last_hidden_state (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`):
            Sequence of hidden-states at the output of the last layer of the model.
        hidden_states (`tuple(torch.FloatTensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `torch.FloatTensor` (one for the output of the embeddings, if the model has an embedding layer, +
            one for the output of each layer) of shape `(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the optional initial embedding outputs.
        attentions (`tuple(torch.FloatTensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `torch.FloatTensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    """
    decoder_logits:Optional[Tuple[torch.FloatTensor]] = None
    image_embeds: Optional[torch.FloatTensor] = None
    vision_last_hidden_state: torch.FloatTensor=None
    encoder_last_hidden_state: torch.FloatTensor = None
    encoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    decoder_last_hidden_state: torch.FloatTensor = None
    decoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None




In [16]:
class customblip_temp(torch.nn.Module):
  def __init__(self,model):
        super(customblip_temp, self).__init__()
        self.model_vis = model.vision_model
        self.model_text_enc = model.text_encoder
        self.model_text_dec = model.text_decoder
        self.decoder_pad_token_id = model.decoder_pad_token_id
        self.decoder_start_token_id = model.decoder_start_token_id
        self.eos_token_id=model.config.text_config.sep_token_id,
        self.pad_token_id=model.config.text_config.pad_token_id
        self.output_attentions=model.config.output_attentions
        self.use_return_dict=model.config.use_return_dict
        self.output_hidden_states=model.config.output_hidden_states

  def image_embed(
      self,
        list_pixel_values: torch.FloatTensor,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
    ):
    image_embeds_tensor=torch.zeros(size=(len(list_pixel_values),577,768))
    for i in range(len(list_pixel_values)):
      vision_outputs = self.model_vis(
              pixel_values=list_pixel_values[i],
              output_attentions=output_attentions,
              output_hidden_states=output_hidden_states,
          )
      image_embeds = vision_outputs[0]
      image_embeds_tensor[i,:,:]=image_embeds[0]

    return image_embeds_tensor
  def forward(
        self,
        input_ids: torch.LongTensor,
        image_embeds: Optional[torch.FloatTensor]=None,
        attention_mask: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, BlipVQAOutput]:

        return_dict = return_dict if return_dict is not None else self.use_return_dict
        output_attentions = output_attentions if output_attentions is not None else self.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.output_hidden_states
        )

        if image_embeds!=None:
          image_embeds=image_embeds.to(device)
          image_embeds_w=image_embeds
          image_attention_mask = torch.ones(image_embeds_w.size()[:-1], dtype=torch.long)

        input_ids=input_ids.to(device)
        question_embeds = self.model_text_enc(
            input_ids=input_ids,
            attention_mask=attention_mask,
            encoder_hidden_states=image_embeds_w,
            encoder_attention_mask=image_attention_mask,
            output_hidden_states=True,
        )

        question_embeds_w = question_embeds[0] if not return_dict else question_embeds.last_hidden_state

        bos_ids = torch.full(
            (question_embeds_w.size(0), 1), fill_value=self.decoder_start_token_id, device=(device)
        )

        answer_output = self.model_text_dec(
            input_ids=bos_ids,
            encoder_hidden_states=question_embeds_w,
            encoder_attention_mask=attention_mask,
            output_hidden_states=True,
            reduction="mean"
        )


        return BlipVQAOutput(
            decoder_logits=answer_output.logits,
            image_embeds=image_embeds_w,
            encoder_last_hidden_state=question_embeds.last_hidden_state,
            encoder_hidden_states=question_embeds.hidden_states,
            decoder_hidden_states=answer_output.hidden_states,
        )

In [17]:
new_model=customblip_temp(model)

In [18]:
new_model.to(device)

customblip_temp(
  (model_vis): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, elementwise_a

In [ ]:
for n,m in new_model.named_modules():
  print(n)


model_vis
model_vis.embeddings
model_vis.embeddings.patch_embedding
model_vis.encoder
model_vis.encoder.layers
model_vis.encoder.layers.0
model_vis.encoder.layers.0.self_attn
model_vis.encoder.layers.0.self_attn.dropout
model_vis.encoder.layers.0.self_attn.qkv
model_vis.encoder.layers.0.self_attn.projection
model_vis.encoder.layers.0.layer_norm1
model_vis.encoder.layers.0.mlp
model_vis.encoder.layers.0.mlp.activation_fn
model_vis.encoder.layers.0.mlp.fc1
model_vis.encoder.layers.0.mlp.fc2
model_vis.encoder.layers.0.layer_norm2
model_vis.encoder.layers.1
model_vis.encoder.layers.1.self_attn
model_vis.encoder.layers.1.self_attn.dropout
model_vis.encoder.layers.1.self_attn.qkv
model_vis.encoder.layers.1.self_attn.projection
model_vis.encoder.layers.1.layer_norm1
model_vis.encoder.layers.1.mlp
model_vis.encoder.layers.1.mlp.activation_fn
model_vis.encoder.layers.1.mlp.fc1
model_vis.encoder.layers.1.mlp.fc2
model_vis.encoder.layers.1.layer_norm2
model_vis.encoder.layers.2
model_vis.encoder

In [ ]:
import requests

In [19]:
def make_inputs(ModelandProcessor,new_model,image,question, device=device):
  d_temp={}
  model=ModelandProcessor.model
  processor=ModelandProcessor.processor

  input_ids = processor(text=question, return_tensors='pt').input_ids.to(device)
  #input_ids = (torch.tensor(input_ids).unsqueeze(0)).to(device)

  pixel_values=processor(images=image,return_tensors="pt")
  pixel_values_1=pixel_values.pixel_values.to(device)
  pixel_values_2=pixel_values.pixel_values.to(device)
  l=[pixel_values_1,pixel_values_2]

  image_embeds=new_model.image_embed(list_pixel_values=l)
  image_embeds.to(device)

  tup=(image_embeds,input_ids)
  return tup



In [ ]:
tup=make_inputs(mt,new_model,image,question)

In [ ]:
tup[0]

In [20]:
def predict_from_input(new_model,tup):
    image_embeds,input_ids=tup
    output=new_model.forward(image_embeds=image_embeds.to(device),input_ids=input_ids.to(device))


    #attention_mask=processor(text=text_input, add_special_tokens=False).attention_mask

    return output

In [21]:
def result_gen(output, return_p=False):
    out=output['decoder_logits']
    probs = torch.softmax(out[:, -1], dim=1)
    p, preds = torch.max(probs, dim=1)
    return preds, p

In [22]:
def decoding(mt,pred_prob):
  preds=pred_prob[0]
  ans=[]
  for i in range(preds.size(0)):
    single_pred=preds[i]
    decoded_answer=processor.decode(single_pred, skip_special_tokens=True)
    ans.append(decoded_answer)
  return ans[0]



In [23]:
def prng(shape: tuple, uniform_noise: bool = True) -> np.ndarray:
      rs = np.random.RandomState(1)
      if uniform_noise:
        return rs.uniform(-1, 1, shape)
      else:
        return rs.randn(*shape)

In [24]:
def noisy_embedding(image_embed_tensor,uniform_noise=True,noise=0.7691,percentage_dimensions=1):
  #num_dimensions_to_corrupt=round(percentage_dimensions*(inp[0].shape[0]))
  corrupt_indices = [i for i in range(0,image_embed_tensor.size(1))]
  if isinstance(noise, float):
        noise_fn = lambda inp: noise * inp
  else:
        noise_fn = noise


  noise_data = torch.rand(size=[1,577,768])
  corrupt_run=image_embed_tensor[1][:,:]
  image_embed_tensor[1]=noise_fn(image_embed_tensor[1])
  image_embed_tensor.to(device)
  return image_embed_tensor

In [ ]:
noise_added_embed

tensor([[[ 0.8730, -0.1344,  0.1977,  ..., -0.2741,  0.2352,  0.7188],
         [-0.2848,  0.2965, -0.2212,  ...,  0.1181,  0.0483, -0.0484],
         [-0.2919,  0.1136, -0.6004,  ...,  1.0594,  0.1805,  0.2552],
         ...,
         [-0.5395,  0.2122, -0.0217,  ...,  0.8717,  0.0089,  0.3548],
         [-0.3715, -0.3431,  0.4678,  ...,  0.1776, -0.3779,  0.0218],
         [ 0.3669, -0.4727,  0.1220,  ..., -0.1135, -0.4058,  0.3387]],

        [[ 0.7836, -0.1207,  0.1774,  ..., -0.2460,  0.2111,  0.6452],
         [-0.2556,  0.2662, -0.1986,  ...,  0.1060,  0.0434, -0.0434],
         [-0.2620,  0.1020, -0.5389,  ...,  0.9509,  0.1620,  0.2291],
         ...,
         [-0.4843,  0.1904, -0.0195,  ...,  0.7824,  0.0080,  0.3185],
         [-0.3335, -0.3080,  0.4199,  ...,  0.1594, -0.3392,  0.0196],
         [ 0.3293, -0.4243,  0.1095,  ..., -0.1019, -0.3642,  0.3040]]],
       grad_fn=<CopySlices>)

In [25]:
def trace_with_patch(
    mt,new_model,  # The model
    image,
    question,  # A set of inputs # A list of (token index, layername) triples to restore
    pred_ans,
    patching_layers,  # Answer probabilities to collect
    replace=False,  # True to replace with instead of add noise
    trace_layers=None,  # List of traced outputs to return
):
    """
    Runs a single causal trace.  Given a model and a batch input where
    the batch size is at least two, runs the batch in inference, corrupting
    a the set of runs [1...n] while also restoring a set of hidden states to
    the values from an uncorrupted run [0] in the batch.

    The convention used by this function is that the zeroth element of the
    batch is the uncorrupted run, and the subsequent elements of the batch
    are the corrupted runs.  The argument tokens_to_mix specifies an
    be corrupted by adding Gaussian noise to the embedding for the batch
    inputs other than the first element in the batch.  Alternately,
    subsequent runs could be corrupted by simply providing different
    input tokens via the passed input batch.

    Then when running, a specified set of hidden states will be uncorrupted
    by restoring their values to the same vector that they had in the
    zeroth uncorrupted run.  This set of hidden states is listed in
    states_to_patch, by listing [(token_index, layername), ...] pairs.
    To trace the effect of just a single state, this can be just a single
    token/layer pair.  To trace the effect of restoring a set of states,
    any number of token indices and layers can be listed.
    """

    #rs = numpy.random.RandomState(1)  # For reproducibility, use pseudorandom noise
    image_embed_tensor,input_ids=make_inputs(mt,new_model,image=image,question=question)
    new_image_embed_tensor=noisy_embedding(image_embed_tensor)

    patch_spec = defaultdict(list)
    for t, l in patching_layers:
        patch_spec[l].append(t)

    #clean_image_attention_mask = torch.ones(clean_inp.size()[:-1], dtype=torch.long)
    #corrup_image_attention_mask=torch.ones(corrupt_inp.size()[:-1], dtype=torch.long)
    def untuple(x):
        return x[0] if isinstance(x, tuple) else x

    def patch_rep(x,layer):
      if layer not in patch_spec:
        return x
      else:
        if untuple(x).size(0)==1:
          return x
        else:
          for t in patch_spec[layer]:
              h=untuple(x)
              h[1:, t,:] = h[0,t,:]
          return x


    # With the patching rules defined, run the patched model in inference.
    #additional_layers = [] if trace_layers is None else trace_layers
    with torch.no_grad(), TraceDict(
        new_model,
        list(patch_spec.keys()),
        edit_output=patch_rep,
    ) as td:
        outputs_exp = new_model.forward(image_embeds=new_image_embed_tensor,input_ids=input_ids)




    # We report softmax probabilities for the answers_t token predictions of interest.
    probs = torch.softmax(outputs_exp.decoder_logits[1:, -1, :], dim=1).mean(dim=0)[pred_ans]
    # If tracing all layers, collect all activations together to return.
    #if trace_layers is not None:
        #all_traced = torch.stack(
         #   [untuple(td[layer].output).detach().cpu() for layer in trace_layers], dim=2
        #)

    return probs


In [26]:
def trace_important_states(
    mt,new_model,start,
    num_layers,block_name,
    image,question,
    pred_ans,num=None,
    replace=False,
):
    table = []
    image_embed_tensor,input_ids=make_inputs(mt,new_model,image=image,question=question)
    if num==None:
      num=input_ids.shape[1]
    for tnum in range(num):
        row = []
        for layer in range(start,num_layers):
            r = trace_with_patch(
                mt,new_model,  # The model
                image,
                question,  # A set of inputs # A list of (token index, layername) triples to restore
                pred_ans,
                [(tnum, layername(new_model, layer,block_name))],
            )
            row.append(r)
        table.append(torch.stack(row))
    return torch.stack(table)


In [50]:
def trace_important_window(
    mt,new_model,start,
    num_layers,block_name,
    image,question,pred_ans,
    kind,
    window=3,num=None,
    replace=False
):
    table = []
    image_embed_tensor,input_ids=make_inputs(mt,new_model,image=image,question=question)
    if num==None:
      num=input_ids.shape[1]
    for tnum in range(num):
        row = []
        for layer in range(start,num_layers):
            layerlist = [
                (tnum, layername(new_model, L,block_name, kind))
                for L in range(
                    max(0, layer - window // 2), min(num_layers, layer - (-window // 2))
                )
            ]
            r = trace_with_patch(
                mt,new_model,  # The model
                image,
                question,  # A set of inputs # A list of (token index, layername) triples to restore
                pred_ans,
                layerlist,
            )
            row.append(r)
        table.append(torch.stack(row))
    return torch.stack(table)


In [29]:
def calculate_hidden_flow(mt,new_model,image,question,block_name,kind=None,expect=None):
     tup=make_inputs(mt,new_model,image=image,question=question)
     output=predict_from_input(new_model,tup)
     pred_prob=result_gen(output)
     new_image_embed_tensor=noisy_embedding(tup[0])
     pred_ans_tensor=pred_prob[0][1]

     predicted_ans=decoding(mt,pred_prob)
     low_score = trace_with_patch(
        mt,new_model, image,question,pred_ans_tensor, []
    ).item()

     if block_name=='text_encoder' and kind==None:
        differences = trace_important_states(
            mt,new_model,0,
            12,block_name,
            image,question,
            pred_ans_tensor
        )
     elif block_name=='text_decoder' and kind==None:
        differences = trace_important_states(
            mt,new_model,1,
            12,block_name,
            image,question,
            pred_ans_tensor,1
        )
     elif block_name=='text_encoder' and kind=='attention':
        differences = trace_important_window(
            mt,new_model,0,
            12,block_name,
            image,question,
            pred_ans_tensor,kind
        )
     elif block_name=='text_encoder' and kind=='crossattention':
        differences = trace_important_window(
            mt,new_model,0,
            12,block_name,
            image,question,
            pred_ans_tensor,kind
        )
     differences = differences.detach().cpu()
     return dict(
        scores=differences,
        low_score=low_score,
        high_score=pred_prob[1],
        question=question,
        answer=predicted_ans,
        correct_prediction=True,
        block_name=block_name,kind=kind
    )









In [45]:
def plot_hidden_flow(
    mt,
    new_model,
    image,question,
    block_name,
    savepdf=None,
):
    result = calculate_hidden_flow(mt,new_model,image,question,block_name)
    plot_trace_heatmap(result, savepdf)

temp_diff=[]
temp_lows=[]
def plot_trace_heatmap(result, savepdf=None, title=None, xlabel=None, modelname=None):
    differences = result["scores"]
    low_score = result["low_score"]
    answer = result["answer"]
    block_name=(str(result["block_name"]))
    #print(differences)
    #print(low_score)
    #temp_diff.append(differences)
    #temp_lows.append(low_score)
   #kind = (
    #    None
     #   if (not result["kind"] or result["kind"] == "None")
      #  else str(result["kind"])
    #)
    #window = result.get("window", 10)
   #labels = list(result["input_tokens"])
    #for i in range(*result["subject_range"]):
     #   labels[i] = labels[i] + "*"
    labels = list(question.split())
    labels.insert(0,'[start]')
    labels.append('[end]')
    labels_news=['[decode]']
    with plt.rc_context(rc={"font.family": "Liberation Serif"}):
        fig, ax = plt.subplots(figsize=(3.5, 2), dpi=200)
        h = ax.pcolor(
            differences,
            cmap={ "text_encoder": "Purples", "text_decoder": "Reds"}[block_name
            ],
            vmin=low_score,
        )
        print(differences.shape)
        ax.invert_yaxis()
        ax.set_yticks([0.5 + i for i in range(len(differences))])
        ax.set_xticks([0.5 + i for i in range(0, differences.shape[1] - 1, 1)])
        ax.set_xticklabels(list(range(0, differences.shape[1] - 1, 1)))
        if not modelname:
            modelname = "BLIP"
        if block_name!=None:
            if block_name=='text_encoder':
              ax.set_yticklabels(labels)
              blockname='text_encoder'
              ax.set_title(f"Impact of restoring {blockname} after corrupted input")
            else:
              blockname='text_decoder'
              ax.set_yticklabels(labels_news)
              ax.set_title(f"Impact of restoring {blockname} after corrupted input")

            #ax.set_xlabel(f"center of interval of {window} restored {kindname} layers")
        cb = plt.colorbar(h)
        if title is not None:
            ax.set_title(title)
        if xlabel is not None:
            ax.set_xlabel(xlabel)
        elif answer is not None:
            # The following should be cb.ax.set_xlabel, but this is broken in matplotlib 3.5.1.
            cb.ax.set_title(f"p({str(answer).strip()})", y=-0.16, fontsize=10)
        if savepdf:
            os.makedirs(os.path.dirname(savepdf), exist_ok=True)
            plt.savefig(savepdf, bbox_inches="tight")
            plt.close()
        else:
            plt.show()


def plot_all_flow(mt,new_model, image,question,savepdf,subject=None):
    for block_name in ['text_decoder']:
        plot_hidden_flow(mt,new_model,image,question,block_name,savepdf)

In [53]:
def plot_kind_hidden_flow(
    mt,
    new_model,
    image,question,
    block_name,kind,
    savepdf=None,
):
    result = calculate_hidden_flow(mt,new_model,image,question,block_name,kind)
    plot_kindtrace_heatmap(result, savepdf)

temp_diff=[]
temp_lows=[]
def plot_kindtrace_heatmap(result, savepdf=None, title=None, xlabel=None, modelname=None):
    differences = result["scores"]
    low_score = result["low_score"]
    answer = result["answer"]
    block_name=(str(result["block_name"]))
    kind=(str(result['kind']))
    #print(differences)
    #print(low_score)
    #temp_diff.append(differences)
    #temp_lows.append(low_score)
   #kind = (
    #    None
     #   if (not result["kind"] or result["kind"] == "None")
      #  else str(result["kind"])
    #)
    #window = result.get("window", 10)
   #labels = list(result["input_tokens"])
    #for i in range(*result["subject_range"]):
     #   labels[i] = labels[i] + "*"
    labels = list(question.split())
    labels.insert(0,'[start]')
    labels.append('[end]')
    labels_news=['[decode]']
    with plt.rc_context(rc={"font.family": "Liberation Serif"}):
        fig, ax = plt.subplots(figsize=(3.5, 2), dpi=200)
        h = ax.pcolor(
            differences,
            cmap={ "attention": "Greens", "crossattention": "Blues"}[kind
            ],
            vmin=low_score,
        )
        print(differences.shape)
        ax.invert_yaxis()
        ax.set_yticks([0.5 + i for i in range(len(differences))])
        ax.set_xticks([0.5 + i for i in range(0, differences.shape[1] - 1, 1)])
        ax.set_xticklabels(list(range(0, differences.shape[1] - 1, 1)))
        if not modelname:
            modelname = "BLIP"
        if block_name!=None and kind!=None:
            if block_name=='text_encoder' and kind=='attention':
              print("attention:",differences)
              ax.set_yticklabels(labels)
              blockname='text_encoder'
              ax.set_title(f"Impact of restoring {kind} layers in {blockname} after corrupted input")
            else:
              blockname='text_encoder'
              kind=='crossattention'
              print("crossattention:",differences)
              ax.set_yticklabels(labels)
              ax.set_title(f"Impact of restoring {kind} layers in {blockname} after corrupted input")

            #ax.set_xlabel(f"center of interval of {window} restored {kindname} layers")
        cb = plt.colorbar(h)
        if title is not None:
            ax.set_title(title)
        if xlabel is not None:
            ax.set_xlabel(xlabel)
        elif answer is not None:
            # The following should be cb.ax.set_xlabel, but this is broken in matplotlib 3.5.1.
            cb.ax.set_title(f"p({str(answer).strip()})", y=-0.16, fontsize=10)
        if savepdf:
            os.makedirs(os.path.dirname(savepdf), exist_ok=True)
            plt.savefig(savepdf, bbox_inches="tight")
            plt.close()
        else:
            plt.show()


def plotkind_all_flow(mt,new_model, image,question,savepdf=None,subject=None):
    for kind in [('attention','/content/causal_trace_sampleexample2_Question Encoder_Attention Layers.pdf'),('crossattention','/content/causal_trace_sampleexample2_Question Encoder_CrossAttention Layers.pdf')]:
        plot_kind_hidden_flow(mt,new_model,image,question,'text_encoder',kind[0],kind[1])

#Test on Pre-trained Model

In [ ]:
!apt-get install -y font-manager
!cp times new roman.ttf /usr/share/fonts/truetype/
!fc-cache -f -v

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  font-manager-common font-viewer
Suggested packages:
  file-roller nautilus-font-manager nemo-font-manager yelp
The following NEW packages will be installed:
  font-manager font-manager-common font-viewer
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,740 kB of archives.
After this operation, 9,394 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 font-manager-common amd64 0.7.7-0.3 [1,545 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 font-viewer amd64 0.7.7-0.3 [27.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 font-manager amd64 0.7.7-0.3 [168 kB]
Fetched 1,740 kB in 1s (2,276 kB/s)
Selecting previously unselected package font-manager-common:amd64.
(Reading database ... 123069 files and directories currently installed.

In [ ]:
import matplotlib.font_manager as fm

font_list = fm.findSystemFonts()
font_names = [fm.get_font(font).family_name for font in font_list]
print(font_names)

['Liberation Sans Narrow', 'Liberation Sans Narrow', 'Liberation Sans', 'Liberation Sans', 'Liberation Serif', 'Liberation Mono', 'Liberation Sans Narrow', 'Liberation Sans', 'Liberation Serif', 'Liberation Sans Narrow', 'Liberation Serif', 'Humor Sans', 'Liberation Mono', 'Liberation Mono', 'Liberation Mono', 'Liberation Serif', 'Liberation Sans']


In [ ]:
main_df=pd.read_csv('/content/clevr_vqa_dataset_3.csv')

In [ ]:
main_df.head()

,Unnamed: 0,Image_IDs,Questions_Answers
0,0,CLEVR_val_000145,[{'question': 'Is the large red cylinder right...
1,1,CLEVR_val_000187,[{'question': 'Is the large green cylinder rig...
2,2,CLEVR_val_000194,[{'question': 'Is the small purple sphere righ...
3,3,CLEVR_val_000206,[{'question': 'Is the large cyan cube right of...
4,4,CLEVR_val_000233,[{'question': 'Is the small brown cylinder rig...


In [41]:
color_df

,Unnamed: 0,final_image_id,train_image_id_count,question_count,answer_count
0,0,000000023004,23004,what is the color of the horses,brown
1,1,000000220218,220218,what is the color of the character,purple
2,2,000000491525,491525,what is the color of the dog,black
3,3,000000256565,256565,what is the color of the sign,red
4,4,000000351203,351203,what is the color of the leash,orange
...,...,...,...,...,...
13108,13108,000000463172,463172,what is the color of the bird,white
13109,13109,000000138368,138368,what is the color of the court,blue
13110,13110,000000381027,381027,what is the color of the inside,purple
13111,13111,000000411815,411815,what is the color of the field,green


In [35]:
image_id=color_df['final_image_id'][1]
url=f"http://images.cocodataset.org/train2017/{image_id}.jpg"
image=Image.open(requests.get(url, stream=True).raw)
question=color_df['question_count'][1]

In [ ]:
temp_diff

[tensor([[0.8839, 0.8839, 0.8839, 0.8839, 0.8839, 0.8838, 0.8838, 0.8838, 0.8837,
          0.8838, 0.8839, 0.8841],
         [0.8839, 0.8840, 0.8840, 0.8840, 0.8840, 0.8839, 0.8838, 0.8838, 0.8838,
          0.8838, 0.8839, 0.8841],
         [0.8839, 0.8839, 0.8839, 0.8839, 0.8839, 0.8838, 0.8838, 0.8838, 0.8837,
          0.8838, 0.8838, 0.8841],
         [0.8839, 0.8839, 0.8839, 0.8839, 0.8838, 0.8838, 0.8838, 0.8838, 0.8837,
          0.8838, 0.8838, 0.8841],
         [0.8839, 0.8838, 0.8838, 0.8835, 0.8838, 0.8837, 0.8839, 0.8839, 0.8838,
          0.8839, 0.8839, 0.8841],
         [0.8839, 0.8839, 0.8840, 0.8840, 0.8840, 0.8838, 0.8838, 0.8838, 0.8838,
          0.8838, 0.8839, 0.8841],
         [0.8840, 0.8840, 0.8840, 0.8840, 0.8840, 0.8838, 0.8838, 0.8838, 0.8838,
          0.8838, 0.8839, 0.8841],
         [0.8839, 0.8836, 0.8837, 0.8840, 0.8847, 0.8844, 0.8842, 0.8838, 0.8839,
          0.8840, 0.8842, 0.8842],
         [0.8839, 0.8839, 0.8839, 0.8839, 0.8839, 0.8839, 0.8839

In [54]:
plotkind_all_flow(mt,new_model,image,question)

torch.Size([9, 12])
attention: tensor([[0.7348, 0.7348, 0.7348, 0.7349, 0.7351, 0.7342, 0.7349, 0.7361, 0.7377,
         0.7377, 0.7452, 0.7451],
        [0.7347, 0.7350, 0.7349, 0.7354, 0.7359, 0.7353, 0.7350, 0.7355, 0.7369,
         0.7369, 0.7448, 0.7446],
        [0.7348, 0.7348, 0.7347, 0.7347, 0.7351, 0.7346, 0.7345, 0.7357, 0.7371,
         0.7370, 0.7450, 0.7448],
        [0.7347, 0.7347, 0.7347, 0.7348, 0.7351, 0.7348, 0.7349, 0.7364, 0.7378,
         0.7378, 0.7456, 0.7455],
        [0.7346, 0.7336, 0.7344, 0.7347, 0.7351, 0.7353, 0.7354, 0.7369, 0.7376,
         0.7377, 0.7452, 0.7448],
        [0.7347, 0.7346, 0.7347, 0.7349, 0.7350, 0.7344, 0.7346, 0.7361, 0.7374,
         0.7376, 0.7456, 0.7456],
        [0.7348, 0.7348, 0.7348, 0.7351, 0.7352, 0.7347, 0.7349, 0.7365, 0.7379,
         0.7380, 0.7459, 0.7458],
        [0.7349, 0.7371, 0.7381, 0.7358, 0.7354, 0.7364, 0.7376, 0.7389, 0.7382,
         0.7368, 0.7430, 0.7426],
        [0.7348, 0.7348, 0.7348, 0.7348, 0.7348, 

In [47]:
plot_all_flow(mt,new_model,image,question,'/content/causaltrace_sampleexample2_Answer Decoder.pdf')

torch.Size([1, 11])


In [ ]:
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [60]:
def plot_kindavg_hidden_flow(
    mt,
    new_model,
    image,question,
    block_name,kind,
    savepdf=None,
):
    result = calculate_hidden_flow(mt,new_model,image,question,block_name,kind)
    return result

In [32]:
color_df=pd.read_csv('/content/color_coco_qa.csv')

In [33]:
def convert_to_id(df):
  temp=df['train_image_id_count']
  df.drop(['Unnamed: 0'],axis=1)
  image_id=[]
  for i in temp:
    str_i=str(i)
    num_of_zeros=12-len(str_i)
    temp_s=""
    for j in range(num_of_zeros):
      temp_s+="0"
    temp_s+=str_i
    image_id.append(temp_s)
  df.insert(loc=1,column = 'final_image_id',
          value = image_id)
  return df

In [34]:
color_df=convert_to_id(color_df)

In [ ]:
def element_wise_average(*tensors):
    num_tensors = len(tensors)
    # Create an empty tensor of the same size as the input tensors
    average_tensor = np.zeros_like(tensors[0])

    # Sum the corresponding elements of the input tensors
    for tensor in tensors:
        average_tensor += tensor

    # Divide the summed tensor by the total number of input tensors
    average_tensor /= num_tensors

    return average_tensor

In [ ]:
ques=color_df['question_count'][25]

In [55]:
color_df

,Unnamed: 0,final_image_id,train_image_id_count,question_count,answer_count
0,0,000000023004,23004,what is the color of the horses,brown
1,1,000000220218,220218,what is the color of the character,purple
2,2,000000491525,491525,what is the color of the dog,black
3,3,000000256565,256565,what is the color of the sign,red
4,4,000000351203,351203,what is the color of the leash,orange
...,...,...,...,...,...
13108,13108,000000463172,463172,what is the color of the bird,white
13109,13109,000000138368,138368,what is the color of the court,blue
13110,13110,000000381027,381027,what is the color of the inside,purple
13111,13111,000000411815,411815,what is the color of the field,green


In [56]:
img_id_arr=[]
answer_arr=[]
question_arr=[]
for i in range(len(color_df['question_count'])):
  f_i_i=color_df['final_image_id'][i]
  answer=color_df['answer_count'][i]
  ques=color_df['question_count'][i]
  if len(ques.split())==7:
    question_arr.append(ques)
    answer_arr.append(answer)
    img_id_arr.append(f_i_i)




In [57]:
color_df=pd.DataFrame()

In [58]:
color_df['final_image_id']=img_id_arr
color_df['question_count']=question_arr
color_df['answer_count']=answer_arr

In [59]:
color_df

,final_image_id,question_count,answer_count
0,000000023004,what is the color of the horses,brown
1,000000220218,what is the color of the character,purple
2,000000491525,what is the color of the dog,black
3,000000256565,what is the color of the sign,red
4,000000351203,what is the color of the leash,orange
...,...,...,...
13057,000000463172,what is the color of the bird,white
13058,000000138368,what is the color of the court,blue
13059,000000381027,what is the color of the inside,purple
13060,000000411815,what is the color of the field,green


In [ ]:
diff_text_enc=0
low_s_text_enc=0
diff_text_dec=0
low_s_text_dec=0
count=500
c1=0
c2=0
for i in range(count):
  image_id=color_df['final_image_id'][i]
  url=f"http://images.cocodataset.org/train2017/{image_id}.jpg"
  image=Image.open(requests.get(url, stream=True).raw)
  question=color_df['question_count'][i]
  #question=question.rstrip()
  result_temp=plot_kindavg_hidden_flow(mt,new_model,image,question,'text_encoder','attention')
  #print(result_temp['scores'])
  try:
    diff_text_enc+=result_temp['scores']
    low_s_text_enc+=result_temp['low_score']
  except:
    c1+=1
    continue

  result_temp_1=plot_kindavg_hidden_flow(mt,new_model,image,question,'text_encoder','crossattention')
  try:
    diff_text_dec+=result_temp_1['scores']
    low_s_text_dec+=result_temp_1['low_score']
  except:
    c2+=1
    continue

In [ ]:
print(c1,c2)

34 0


In [ ]:
print(i)

25


In [ ]:
diff_text_dec

tensor([[0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8443, 0.8452,
         0.8455, 0.8465]])

In [ ]:
diff_text_enc=diff_text_enc/(count-34)
low_s_text_enc=low_s_text_enc/(count-34)
diff_text_dec=diff_text_dec/(count-34)
low_s_text_dec=low_s_text_dec/(count-34)

In [ ]:
diff_text_enc

tensor([[0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8443, 0.8444, 0.8445,
         0.8446, 0.8449, 0.8449],
        [0.8442, 0.8442, 0.8442, 0.8443, 0.8443, 0.8442, 0.8442, 0.8443, 0.8444,
         0.8444, 0.8448, 0.8447],
        [0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8441, 0.8441, 0.8442, 0.8444,
         0.8443, 0.8447, 0.8446],
        [0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8441, 0.8442, 0.8443, 0.8444,
         0.8444, 0.8448, 0.8447],
        [0.8443, 0.8443, 0.8443, 0.8443, 0.8443, 0.8443, 0.8443, 0.8444, 0.8445,
         0.8444, 0.8448, 0.8447],
        [0.8442, 0.8442, 0.8442, 0.8442, 0.8443, 0.8441, 0.8442, 0.8443, 0.8444,
         0.8445, 0.8448, 0.8448],
        [0.8442, 0.8442, 0.8442, 0.8442, 0.8443, 0.8441, 0.8442, 0.8443, 0.8445,
         0.8445, 0.8448, 0.8448],
        [0.8441, 0.8443, 0.8443, 0.8441, 0.8446, 0.8445, 0.8442, 0.8455, 0.8452,
         0.8451, 0.8453, 0.8450],
        [0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8442, 0.8442,

In [ ]:
result={'differences_TE':diff_text_enc,'low_score_TE':low_s_text_enc,'differences_DE':diff_text_dec,'low_score_DE':low_s_text_dec}

In [ ]:
def avgnormal_heatmap(result,block_name,kind, savepdf=None, title=None, xlabel=None, modelname=None):
  if block_name=='Question Encoder' and kind=='Attention':
    avg_differences=result['differences_TE']
    avg_low_score=result['low_score_TE']
    answer = "QE"
    labels=['[Encode]','Q_token_1','Q_token_2','Q_token_3','Q_token_4','Q_token_5','Q_token_6','Q_token_7','[End]']
    block_name='text_encoder'
  elif block_name=='Question Encoder' and kind=='CrossAttention':
    avg_differences=result['differences_DE']
    avg_low_score=result['low_score_DE']
    answer = "ANS"
    labels=['Decode']
    block_name='text_encoder'
  normalize_diffs=avg_differences-avg_low_score

  with plt.rc_context(rc={"font.family": "Liberation Serif"}):
        fig, ax = plt.subplots(figsize=(3.5, 2), dpi=200)
        h = ax.pcolor(
            normalize_diffs,
            cmap={ "attention": "Greens",'crossattention':"Blues"}[kind
            ],
            vmin=0.0,
        )
        ax.invert_yaxis()
        ax.set_yticks([0.5 + i for i in range(len(normalize_diffs))])
        ax.set_xticks([0.5 + i for i in range(0, normalize_diffs.shape[1] - 1, 1)])
        ax.set_xticklabels(list(range(0, normalize_diffs.shape[1] - 1, 1)))
        if not modelname:
            modelname = "BLIP"
        if block_name!=None:
              ax.set_yticklabels(labels)
              ax.set_title(f"Impact of restoring {kind} layers in {block_name} after corrupted input")
            #ax.set_xlabel(f"center of interval of {window} restored {kindname} layers")
        cb = plt.colorbar(h)
        if title is not None:
            ax.set_title(title)
        if xlabel is not None:
            ax.set_xlabel(xlabel)
        elif answer is not None:
            # The following should be cb.ax.set_xlabel, but this is broken in matplotlib 3.5.1.
            cb.ax.set_title(f"p({answer})", y=-0.16, fontsize=10)
        if savepdf:
            #os.makedirs(os.path.dirname(savepdf), exist_ok=True)
            plt.savefig(savepdf, bbox_inches="tight")
            plt.close()
        else:
            plt.show()

In [ ]:
avgnormal_heatmap(result,'Question Encoder','/content/466samples_normalized_causal_trace_Question_Encoder.pdf')

In [ ]:
avgnormal_heatmap(result,'Answer Decoder','/content/466samples_normalized_causal_trace_Answer_Decoder.pdf')

In [ ]:
high_score = None  # Scale all plots according to the y axis of the first plot

for block_name in ['text_encoder','text_decoder']:
    d = read_knowlege(6,block_name)
    count = d["size"]
    what = {
        "text_encoder": "Indirect Effect in Question Encoder",
        "text_decoder": "Indirect Effect in Answer Decoder",
    }[block_name]
    title = f"Avg {what} over {count} prompts"
    result = numpy.clip(d["result"] - d["low_score"], 0, None)
    #kindcode = "" if kind is None else f"_{kind}"
    if block_name not in ["text_encoder", "text_decoder"]:
        high_score = result.max()
    plot_array(
        result,
        block_name,
        title=title,
        low_score=0.0,
        high_score=high_score
    )